In [2]:
import math
import random
import pandas as pd
import requests
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

# Define your Mapbox token
MAPBOX_TOKEN = 'pk.eyJ1IjoiZGFuaWVsYWthbGFtdWRvIiwiYSI6ImNseW4wNno4bTAxNDAya3M0YjJqNHkwamMifQ.RXniMT8_Seus5fdPUJ2XRA'

# Average speeds in km/h for different modes
speeds = {
    "driving": 60,
    "driving-traffic": 50,
    "walking": 5,
    "cycling": 15
}

# Calories burned per hour
calories_burned_per_hour = {
    "driving": 0,
    "driving-traffic": 0,
    "walking": 300,
    "cycling": 600
}

# Carbon footprint in grams of CO2 per kilometer
carbon_footprint_per_km = {
    "driving": 170,
    "driving-traffic": 150,
    "walking": 0,
    "cycling": 0
}

# Function to get coordinates from location names
def geocode_location(location):
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{location}.json?access_token={MAPBOX_TOKEN}"
    response = requests.get(url)
    data = response.json()
    
    if len(data['features']) == 0:
        print(f"Location '{location}' not found. Please check the spelling or try a more specific location.")
        return None

    return data['features'][0]['center']

def get_routes(start_coords, end_coords, mode):
    valid_modes = ['driving', 'walking', 'cycling','driving-traffic']
    if mode not in valid_modes:
        raise ValueError(f"Invalid mode. Choose from {valid_modes}")

    url = f"https://api.mapbox.com/directions/v5/mapbox/{mode}/{start_coords[0]},{start_coords[1]};{end_coords[0]},{end_coords[1]}?geometries=geojson&alternatives=true&access_token={MAPBOX_TOKEN}"
    
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Mapbox API error: {response.status_code}, {response.text}")
    
    routes = response.json().get('routes', [])
    
    return routes

# Function to process routes into a dataframe
def process_routes(routes):
    data = []
    for idx, route in enumerate(routes):
        data.append({
            'Route Label': f'Route {idx + 1}',
            'Distance (km)': route['distance'] / 1000,  # Convert to kilometers
            'Duration (minutes)': route['duration'] / 60,  # Convert to minutes
        })
    return pd.DataFrame(data)

# Function to generate additional data for optimization
def generate_additional_data(df, mode):
    data = {
        "Distance (km)": [],
        "Time (minutes)": [],
        "Calories Burned": [],
        "Carbon Footprint (g CO2)": [],
        "Noise Pollution (dB)": [],
        "Scenic Score": [],
        "AQI": [],
        "Safety Rating": [],
        "Route Label": []
    }
    
    speed = speeds[mode]
    for _, row in df.iterrows():
        time_hours = row['Distance (km)'] / speed
        data["Distance (km)"].append(row['Distance (km)'])
        data["Time (minutes)"].append(row['Duration (minutes)'])
        data["Calories Burned"].append(round(calories_burned_per_hour[mode] * time_hours))
        data["Carbon Footprint (g CO2)"].append(round(carbon_footprint_per_km[mode] * row['Distance (km)']))
        data["Noise Pollution (dB)"].append(random.randint(20, 70))
        data["Scenic Score"].append(random.randint(1, 10))
        base_aqi = random.randint(10, 60)
        aqi_variation = random.uniform(-0.1, 0.1)
        data["AQI"].append(round(base_aqi * (1 + aqi_variation)))
        data["Safety Rating"].append(random.randint(1, 5))
        data["Route Label"].append(row['Route Label'])

    return pd.DataFrame(data)

# Function to normalize data
def normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value) if max_value != min_value else 0

# Class for multi-objective route optimization
class RouteOptimizationProblem(Problem):
    def __init__(self, routes, selected_features, objectives):
        self.routes = routes
        self.selected_features = selected_features
        self.objectives = objectives
        
        n_var = len(routes[0])  # Number of features (variables)
        n_obj = len(selected_features)  # Number of objectives
        
        xl = np.min(routes, axis=0)
        xu = np.max(routes, axis=0)
        
        super().__init__(n_var=n_var, n_obj=n_obj, n_constr=0, xl=xl, xu=xu)
    
    def _evaluate(self, x, out, *args, **kwargs):
        selected = x[:, self.selected_features]
        
        results = []
        for obj in self.objectives:
            if obj == 'min':
                results.append(np.min(selected, axis=1))
            elif obj == 'max':
                results.append(np.max(selected, axis=1))
        
        out["F"] = np.column_stack(results)

# Function to run the optimization and rank routes
def run_optimization(start_location, end_location, mode, selected_features, objectives):
    # Get coordinates
    start_coords = geocode_location(start_location)
    end_coords = geocode_location(end_location)
    
    if start_coords is None or end_coords is None:
        print("Unable to retrieve coordinates for the provided locations.")
        return
    
    # Get routes
    routes = get_routes(start_coords, end_coords, mode)
    
    # Process routes into a dataframe
    df = process_routes(routes)
    
    # Print the original DataFrame
    print("Original DataFrame:")
    print(df)
    
    # Generate additional data based on the selected mode
    additional_data_df = generate_additional_data(df, mode)
    
    # Normalize the data
    additional_data_df["Norm Distance"] = normalize(additional_data_df["Distance (km)"], additional_data_df["Distance (km)"].min(), additional_data_df["Distance (km)"].max())
    additional_data_df["Norm Time"] = normalize(additional_data_df["Time (minutes)"], additional_data_df["Time (minutes)"].min(), additional_data_df["Time (minutes)"].max())
    additional_data_df["Norm Calories"] = normalize(additional_data_df["Calories Burned"], additional_data_df["Calories Burned"].min(), additional_data_df["Calories Burned"].max())
    additional_data_df["Norm Carbon"] = normalize(additional_data_df["Carbon Footprint (g CO2)"], additional_data_df["Carbon Footprint (g CO2)"].min(), additional_data_df["Carbon Footprint (g CO2)"].max())
    additional_data_df["Norm Noise"] = normalize(additional_data_df["Noise Pollution (dB)"], additional_data_df["Noise Pollution (dB)"].min(), additional_data_df["Noise Pollution (dB)"].max())
    additional_data_df["Norm Scenic"] = normalize(additional_data_df["Scenic Score"], additional_data_df["Scenic Score"].min(), additional_data_df["Scenic Score"].max())
    additional_data_df["Norm AQI"] = normalize(additional_data_df["AQI"], additional_data_df["AQI"].min(), additional_data_df["AQI"].max())
    additional_data_df["Norm Safety Rating"] = normalize(additional_data_df["Safety Rating"], additional_data_df["Safety Rating"].min(), additional_data_df["Safety Rating"].max())

    # Print the DataFrame after normalization
    print("\nNormalized DataFrame:")
    print(additional_data_df)
    
    # Prepare data for multi-objective optimization
    routes_data = additional_data_df[[
        "Norm Distance", "Norm Time", "Norm Calories", "Norm Carbon", 
        "Norm Noise", "Norm Scenic", "Norm AQI", "Norm Safety Rating"
    ]].values  # Convert to NumPy array

    # Print the NumPy array before feeding into the model
    print("\nNumPy Array of Routes Data:")
    print(routes_data)
    
    # Create the optimization problem
    problem = RouteOptimizationProblem(routes_data, selected_features, objectives)

    algorithm = NSGA2(pop_size=100)
    
    # Execute the optimization
    res = minimize(problem, algorithm, termination=('n_gen', 30), seed=1, save_history=True, verbose=True)
    
    # Get the optimal solutions and rank the initial routes
    optimal_solutions = res.F
    optimal_routes = res.X
    
    # Calculate the objectives for each initial route based on selected features
    initial_route_objectives = []
    for route in routes_data:
        objectives_values = []
        for idx, obj in zip(selected_features, objectives):
            if obj == 'min':
                objectives_values.append(route[idx])
            elif obj == 'max':
                objectives_values.append(-route[idx])
        initial_route_objectives.append(objectives_values)
    
    initial_route_objectives = np.array(initial_route_objectives)

    # Rank the routes by proximity to the Pareto front
    def compare_to_pareto(route_obj, pareto_objs):
        distances = np.linalg.norm(pareto_objs - route_obj, axis=1)
        return np.min(distances)

    ranked_routes = []
    for i, objectives in enumerate(initial_route_objectives):
        distance_to_pareto = compare_to_pareto(objectives, optimal_solutions)
        ranked_routes.append((i, distance_to_pareto))
    
    ranked_routes.sort(key=lambda x: x[1])

    # Display ranked routes
    print("\nRanked Routes:")
    for rank, (idx, score) in enumerate(ranked_routes):
        print(f"Rank {rank + 1}: Route {idx + 1}, Proximity to Pareto front: {score:.4f}")

# Example call to run the optimization
start_location = "Birmingham"
end_location = "Coventry"
mode = "cycling"
selected_features = [0, 1, 2, 3, 4, 5, 6, 7]  # Adjust based on your features
objectives = ['min', 'min', 'min', 'min', 'min', 'max', 'min', 'max']  # Example objectives

run_optimization(start_location, end_location, mode, selected_features, objectives)

Original DataFrame:
  Route Label  Distance (km)  Duration (minutes)
0     Route 1        34.4863          171.345000
1     Route 2        36.4887          161.155000
2     Route 3        39.3753          164.913333

Normalized DataFrame:
   Distance (km)  Time (minutes)  Calories Burned  Carbon Footprint (g CO2)  \
0        34.4863      171.345000             1379                         0   
1        36.4887      161.155000             1460                         0   
2        39.3753      164.913333             1575                         0   

   Noise Pollution (dB)  Scenic Score  AQI  Safety Rating Route Label  \
0                    37             6   19              4     Route 1   
1                    47             6   13              1     Route 2   
2                    24             5   19              4     Route 3   

   Norm Distance  Norm Time  Norm Calories  Norm Carbon  Norm Noise  \
0       0.000000   1.000000       0.000000            0    0.565217   
1       0